In [1]:
import pickle
from statistics import mean
import numpy as np
import sys

In [2]:
file_name_train_250 = "../Human Experiments/Data/RL_trajectories_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "../Human Experiments/Data/Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "../Human Experiments/Data/RL_trajectories_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "../Human Experiments/Data/Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [3]:
def generate_policies(price_low_bound, price_upper_bound):
    policies = []
    for i in range(price_low_bound, price_upper_bound, 10):
        for j in range(price_low_bound, price_upper_bound, 10):
            for k in range(price_low_bound, price_upper_bound, 10):
                for l in range(5):
                    for m in range(5):
                        for n in range(5):
                            policy = (i, j, k, l, m, n)
                            policies.append(policy)
    return policies

In [14]:
def generate_policies_two_split(price_low_bound, price_upper_bound, step_size):
    policies = []
    for i in range(price_low_bound, price_upper_bound, step_size):
        for j in range(price_low_bound, price_upper_bound, step_size):
            for k in range(8):
                for l in range(7):
                    policy = (i, j, k, l)
                    policies.append(policy)
    return policies

In [37]:
policies = generate_policies_two_split(320, 530, 5)

In [38]:
len(policies)

98784

In [6]:
def evaluate_policy(policy, data, best_buys):
    bought = []
    bought_cost_only = []
    t1 = policy[0]
    t2 = policy[1]
    t3 = policy[2]
    d1 = policy[3]
    d2 = policy[4]
    d3 = policy[5]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            local_cat = int(idx/5)
            local_idx = int(idx%5)
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                break
            elif(local_cat == 0):
                if(cost > t1 and local_idx < d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    break
            elif(local_cat == 1):
                if(cost > t2 and local_idx < d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    break
            elif(local_cat == 2):
                if(cost > t3 and local_idx < d3):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    break
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys)]    
    avg_error = sum(error)/len(data)
    return avg_error

In [20]:
def evaluate_policy_two_split(policy, data, best_buys):
    bought = []
    bought_cost_only = []
    t1 = policy[0]
    t2 = policy[1]
    d1 = policy[2]
    d2 = policy[3]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            if (idx<8):
                local_idx = idx
            else:
                local_idx = idx-8
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                break
            elif(idx < 8):
                if(cost > t1 and local_idx < d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    break
            elif(idx >= 8):
                if(cost > t2 and local_idx < d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    break
            
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys)]    
    avg_error = sum(error)/len(data)
    return avg_error

In [7]:
best_buy_250_train = [min(ele) for ele in trajectories_train_250]
best_buy_250_test = [min(ele) for ele in trajectories_test_250]
best_buy_500_train = [min(ele) for ele in trajectories_train_500]
best_buy_500_test = [min(ele) for ele in trajectories_test_500]

In [54]:
evaluate_policy((190, 210, 230, 5, 5, 3), trajectories_train_250, best_buy_250_train)

17.2033

In [25]:
evaluate_policy_two_split((210, 230, 7, 3), trajectories_train_250, best_buy_250_train)

26.0888

In [26]:
def policy_search(price_low_bound, price_upper_bound, data, best_buys):
    all_errors = []
    all_policies = generate_policies(price_low_bound, price_upper_bound)
    for idx, policy in enumerate(all_policies):
        policy_avg_error = evaluate_policy(policy, data, best_buys)
        sys.stdout.flush()
        sys.stdout.write("\rProgress: {}/{}".format(idx, len(all_policies)))
        all_errors.append(policy_avg_error)
    return(min(all_errors), all_policies[all_policies.index(min(all_errors))])

In [39]:
def policy_search_two_split(price_low_bound, price_upper_bound, data, best_buys):
    all_errors = []
    all_policies = generate_policies_two_split(price_low_bound, price_upper_bound, 5)
    for idx, policy in enumerate(all_policies):
        policy_avg_error = evaluate_policy_two_split(policy, data, best_buys)
        sys.stdout.flush()
        sys.stdout.write("\rProgress: {}/{}".format(idx+1, len(all_policies)))
        all_errors.append(policy_avg_error)
    return(min(all_errors), all_policies[all_errors.index(min(all_errors))])

In [34]:
# optimal_policy = policy_search(160, 260, trajectories_train_250, best_buy_250_train)

In [40]:
optimal_policy_500 = policy_search_two_split(320, 530, trajectories_train_500, best_buy_500_train)

Progress: 98784/98784

In [41]:
optimal_policy_500

(38.1196, (465, 320, 7, 0))